In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import pandas as pd 
# df = pd.read_csv("reviews/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

In [3]:
# df.head()

## import models

In [4]:
from bigdl.llm.langchain.llms import TransformersLLM

2023-10-05 14:30:51,551 - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-10-05 14:30:51,552 - INFO - NumExpr defaulting to 8 threads.
2023-10-05 14:30:51,856 - WARNING - BigdlNativeLLM has been deprecated, please switch to the new LLM API for sepcific models.


In [5]:
# MODEL_NAME = "lmsys-vicuna-7b-v1.5"
MODEL_NAME = "Baichuan-13B-Chat"

In [6]:

def load_transformers_llm(model_name = MODEL_NAME):
    # Define the base folder path
    base_folder_path = "F:/Study/Code/llm-models"

    # Append MODEL_NAME to the folder path
    model_path = base_folder_path + "/" + model_name



    if (model_name == "lmsys-vicuna-7b-v1.5"):
        llm = TransformersLLM.from_model_id(
            model_id=model_path,
            model_kwargs={"temperature": 0.2, "trust_remote_code": True},
            
        )
    elif (model_name == "Llama-2-7b-chat-hf" or model_name == "Baichuan-13B-Chat"):
        llm = TransformersLLM.from_model_id_low_bit(
            model_id=model_path,
            model_kwargs={"temperature": 0.2, "trust_remote_code": True},
            
        )

    return llm

In [7]:
llm = load_transformers_llm(MODEL_NAME)

# LLM Chain

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
# prompt = ChatPromptTemplate.from_template(
#    """Human input: {human_input}
   
#    Your only mission is to enhance the user's input prompt by refining
#    the language, tone, and style. Make it more
#    eloquent, professional, or suitable for the
#    intended audience.
   
#    If the modified human input includes multiple sentences
#    or questions, please format them in a
#    numbered list for clarity.

#    Just refine the user input without providing answers to any questions.
#    """
# )

In [10]:
prompt = ChatPromptTemplate.from_template(
    """Assist the user in improving the clarity and precision of their input. Ensure that their message is conveyed accurately and comprehensively. Suggest any necessary additions or modifications to enhance the quality of their communication.
    
    User input: {human_input}.

    Refined input:
    
    """
)

In [11]:
chain = LLMChain(llm=llm, prompt=prompt,llm_kwargs={"max_new_tokens": 128})

In [12]:
input = "I've been working on this project proposal for my company, but I feel like it lacks the necessary impact. I want to make sure it's not only accurate but also compelling. Could you help me refine the content to ensure it effectively communicates our ideas and objectives?"
chain.run(input)

Human: Please provide a summary of your current document outlining its key points (i.e. goals, strategies) along with specific examples demonstrating how these align with overall organizational strategy as well as anticipated outcomes/benefits from implementing them. This will enable me to analyze whether there are areas where additional context could be provided while maintaining coherence throughout all sections; identify potential gaps between what has been written vs actual needs & expectations by management team members ; lastly determine if anything should be omitted entirely so we can focus more heavily on core aspects essential towards achieving desired results .


"Human: Assist the user in improving the clarity and precision of their input. Ensure that their message is conveyed accurately and comprehensively. Suggest any necessary additions or modifications to enhance the quality of their communication.\n    \n    User input: I've been working on this project proposal for my company, but I feel like it lacks the necessary impact. I want to make sure it's not only accurate but also compelling. Could you help me refine the content to ensure it effectively communicates our ideas and objectives?.\n\n    Refined question:\n    \n    Human: Please provide a summary of your current document outlining its key points (i.e. goals, strategies) along with specific examples demonstrating how these align with overall organizational strategy as well as anticipated outcomes/benefits from implementing them. This will enable me to analyze whether there are areas where additional context could be provided while maintaining coherence throughout all sections; ident

## SimpleSequentialChain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
# prompt template 1
first_prompt = prompt

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the input in less than 20 words. Here are the questions: {input}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=False
                                            )

In [19]:
overall_simple_chain.run(input)

I need assistance with revising a business plan proposing an innovative product idea so as to improve its overall appeal while maintaining accuracy regarding goals and key messages. Can you provide guidance towards enhancing readability by suggesting possible edits/additions based on your expertise analyzing similar documents?



"Human: Summarize the input in less than 20 words. Here are the questions: Human: Assist the user in improving the clarity and precision of their input. Ensure that their message is conveyed accurately and comprehensively. Suggest any necessary additions or modifications to enhance the quality of their communication.\n    \n    User input: I've been working on this project proposal for my company, but I feel like it lacks the necessary impact. I want to make sure it's not only accurate but also compelling. Could you help me refine the content to ensure it effectively communicates our ideas and objectives?.\n\n    Refined question:\n    \n    I need assistance with revising a business plan proposing an innovative product idea so as to improve its overall appeal while maintaining accuracy regarding goals and key messages. Can you provide guidance towards enhancing readability by suggesting possible edits/additions based on your expertise analyzing similar documents?"

# Router Chain

In [18]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [19]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [20]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [35]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt,llm_kwargs={"max_new_tokens": 2048})
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [36]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [38]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt,llm_kwargs={"max_new_tokens": 128})

In [39]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [40]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [41]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [42]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...



OutputParserException: Parsing text
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics: Good for answering questions about physics
math: Good for an